In [ ]:
from itertools import chain
from pathlib import Path

from datasets import concatenate_datasets

from utils.dataset_utils import get_politicalness_datasets, \
    get_politicalness_datasets_from_leaning_datasets
from utils.model_utils import evaluate_models, get_custom_politicalness_models

In [ ]:
GET_DATASETS = lambda: chain(
    get_politicalness_datasets(),
    get_politicalness_datasets_from_leaning_datasets()
)
DATASET_SAMPLE_SIZE = 1_000

datasets = list(
    map(
        lambda dataset: dataset
        .take_even_class_distribution_sample(DATASET_SAMPLE_SIZE)
        .transform_for_inference()
        .to_huggingface(),
        GET_DATASETS(),
    )
)

In [ ]:
# Lambda, so that the generator can be reused.
GET_MODELS = lambda: get_custom_politicalness_models(Path("dataset_benchmark") / "leave_one_in" / "politicalness")
TRUNCATE_TOKENS = True

evaluate_models(
    GET_MODELS,
    datasets,
    TRUNCATE_TOKENS
)

In [ ]:
concatenated_dataset = concatenate_datasets(datasets)

evaluate_models(
    GET_MODELS,
    [concatenated_dataset],
    TRUNCATE_TOKENS
)